# Library imports

In [1]:
import numpy as np
import pandas as pd

from time import time

from xgboost import XGBClassifier

import random as r

# Helper Functions

In [2]:
def breaker():
  print("\n" + 30*"-" + "\n")

def head(x, no_of_ele=5):
  if len(x.shape) > 1:
    breaker()
    print(x[:no_of_ele, :])
    breaker()
  else:
    breaker()
    print(x[:no_of_ele])
    breaker()

def preprocess(x):
  df = x.copy()
  df.iloc[:, 1] = df.iloc[:, 1].map({"trt_cp" : 0, "ctl_vehicle" : 1})
  df.iloc[:, 3] = df.iloc[:, 3].map({"D1" : 0, "D2" : 1})
  return df

def getCol(x):
  return [col for col in x.columns]

# Data Handling and Preprocessing

**Input**

In [3]:
tr_feat = pd.read_csv('../input/lish-moa/train_features.csv')
tr_lbls = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
ts_feat = pd.read_csv('../input/lish-moa/test_features.csv')

breaker()
print("TRAINING FEATURES DATASET SHAPE :", repr(tr_feat.shape))
breaker()
print("TRAINING LABELS DATASET SHAPE   :", repr(tr_lbls.shape))
breaker()
print("TEST FEATURES DATASET SHAPE     :", repr(ts_feat.shape))
breaker()


------------------------------

TRAINING FEATURES DATASET SHAPE : (23814, 876)

------------------------------

TRAINING LABELS DATASET SHAPE   : (23814, 207)

------------------------------

TEST FEATURES DATASET SHAPE     : (3982, 876)

------------------------------



**Top Features Taken From Public Kernel**

In [4]:
top_feats = [  1,   2,   3,   4,   5,   6,   7,   9,  11,  14,  15,  16,  17,
              18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  29,  30,  31,
              32,  33,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  46,
              47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  58,  59,  60,
              61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
              74,  75,  76,  78,  79,  80,  81,  82,  83,  84,  86,  87,  88,
              89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
              102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
              115, 116, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127, 128,
              129, 130, 131, 132, 133, 136, 137, 138, 139, 140, 141, 142, 143,
              144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157,
              158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
              171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
              184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197,
              198, 199, 200, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212,
              213, 214, 215, 216, 217, 218, 219, 220, 221, 223, 224, 225, 226,
              227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239,
              240, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253,
              254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266,
              267, 268, 269, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280,
              281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 294,
              295, 296, 298, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309,
              310, 311, 312, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323,
              324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336,
              337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349,
              350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362,
              363, 364, 365, 366, 367, 368, 369, 370, 371, 374, 375, 376, 377,
              378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 390, 391,
              392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404,
              405, 406, 407, 408, 409, 411, 412, 413, 414, 415, 416, 417, 418,
              419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431,
              432, 434, 435, 436, 437, 438, 439, 440, 442, 443, 444, 445, 446,
              447, 448, 449, 450, 453, 454, 456, 457, 458, 459, 460, 461, 462,
              463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475,
              476, 477, 478, 479, 481, 482, 483, 484, 485, 486, 487, 488, 489,
              490, 491, 492, 493, 494, 495, 496, 498, 500, 501, 502, 503, 505,
              506, 507, 509, 510, 511, 512, 513, 514, 515, 518, 519, 520, 521,
              522, 523, 524, 525, 526, 527, 528, 530, 531, 532, 534, 535, 536,
              538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 549, 550, 551,
              552, 554, 557, 559, 560, 561, 562, 565, 566, 567, 568, 569, 570,
              571, 572, 573, 574, 575, 577, 578, 580, 581, 582, 583, 584, 585,
              586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 599,
              600, 601, 602, 606, 607, 608, 609, 611, 612, 613, 615, 616, 617,
              618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630,
              631, 632, 633, 634, 635, 636, 637, 638, 639, 641, 642, 643, 644,
              645, 646, 647, 648, 649, 650, 651, 652, 654, 655, 656, 658, 659,
              660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672,
              673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685,
              686, 687, 688, 689, 691, 692, 693, 694, 695, 696, 697, 699, 700,
              701, 702, 704, 705, 707, 708, 709, 710, 711, 713, 714, 716, 717,
              718, 720, 721, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732,
              733, 734, 735, 737, 738, 739, 740, 742, 743, 744, 745, 746, 747,
              748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 759, 760, 761,
              762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774,
              775, 776, 777, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788,
              789, 790, 792, 793, 794, 795, 796, 797, 798, 800, 801, 802, 803,
              804, 805, 806, 808, 809, 811, 813, 814, 815, 816, 817, 818, 819,
              821, 822, 823, 825, 826, 827, 828, 829, 830, 831, 832, 834, 835,
              837, 838, 839, 840, 841, 842, 845, 846, 847, 848, 850, 851, 852,
              854, 855, 856, 858, 859, 860, 861, 862, 864, 866, 867, 868, 869,
              870, 871, 872, 873, 874]

**Preprocessing**

In [5]:
X_train = tr_feat.copy()
y_train = tr_lbls.copy()
X_test  = ts_feat.copy()

columns = getCol(tr_lbls)

X_train = preprocess(X_train)
X_test  = preprocess(X_test)

y_train = tr_lbls.loc[X_train['cp_type'] == 0].reset_index(drop=True)
X_train = X_train.loc[X_train['cp_type'] == 0].reset_index(drop=True)

X_train = X_train.iloc[:, top_feats].values
y_train = y_train.iloc[:, 1:].values

sigid  = X_test.iloc[:, 0].values
X_test = X_test.iloc[:, top_feats].values

breaker()
print(repr(X_train.shape))
breaker()
print(repr(X_test.shape))
breaker()
print(repr(y_train.shape))
breaker()

del tr_feat, tr_lbls


------------------------------

(21948, 785)

------------------------------

(3982, 785)

------------------------------

(21948, 206)

------------------------------



# XGB Classification

**Setup**

In [6]:
num_classes  = y_train.shape[1]
num_features = X_train.shape[1]

**Classification**

In [7]:
start_time = time()
final_pred = np.zeros((X_test.shape[0], num_classes))

seeders = [r.randint(0,99) for i in range(5)]
breaker()
for seed in seeders:
    y_pred = np.empty((X_test.shape[0], 1))
    print("Processing Seed {seed} ...".format(seed=seed))
    xgb = [XGBClassifier(n_estimators=100, random_state=seed, tree_method="gpu_hist")] * num_classes
    for i in range(num_classes):
      xgb[i].fit(X_train, y_train[:, i])
      y_hat = xgb[i].predict_proba(X_test)[:, 1]
      y_pred = np.concatenate((y_pred, y_hat.reshape(-1,1)), axis=1)
    y_pred = y_pred[:, 1:]
    final_pred = np.add(y_pred, final_pred)

final_pred = np.divide(final_pred, len(seeders))

breaker()
print("Time taken : {:.2f} minutes".format((time() - start_time)/60))
breaker()


------------------------------

Processing Seed 7 ...
Processing Seed 45 ...
Processing Seed 30 ...
Processing Seed 32 ...
Processing Seed 77 ...

------------------------------

Time taken : 26.15 minutes

------------------------------



# Submission

In [8]:
final_data = np.concatenate((sigid.reshape(-1,1), final_pred), axis=1)

final_df = pd.DataFrame(final_data, columns=columns)
columns = columns[1:]
final_df.loc[ts_feat['cp_type']=='ctl_vehicle', columns] = 0
final_df.to_csv("./submission.csv",index=False)